## The New Bechdel test!
Analysis on the new Bechdel test using the Cornell Movie-Dialog Corpus

In [52]:
import csv
import pandas as pd
import re
from unidecode import unidecode

In [53]:
# Import all the datasets first
# Movie Titles Metadata.txt
mtm = 'cornell movie-dialogs corpus/movie_titles_metadata.txt'
# CSV for transforming the data
mtm_csv = 'mtm_csv.csv'

In [54]:
# Add header rows
with open(mtm_csv, 'wt') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames = ["Movie_Number", "Movie_Name", "Year_of_Release", 
                                                   "IMDB_Rating", "Number_of_Votes", "Genres"])
    writer.writeheader()

In [55]:
# Open the .txt file and write it onto the dataframe
with open(mtm, 'r') as mtmeta:
    for lines in mtmeta:
        text = str(lines)
        line_list = text.split('+++$+++')
        
        with open(mtm_csv, 'a') as write_csv:
            writer = csv.writer(write_csv)
            writer.writerow([line_list[0].strip(), line_list[1].strip(), line_list[2].strip(),
                            line_list[3].strip(), line_list[4].strip(), line_list[5].strip()])

In [56]:
# Check the content of the csv file
mtm_df = pd.read_csv(mtm_csv, index_col = False)
print(mtm_df.shape)
mtm_df.head(5)

(617, 6)


,Movie_Number,Movie_Name,Year_of_Release,IMDB_Rating,Number_of_Votes,Genres
0,m0,10 things i hate about you,1999,6.9,62847,"['comedy', 'romance']"
1,m1,1492: conquest of paradise,1992,6.2,10421,"['adventure', 'biography', 'drama', 'history']"
2,m2,15 minutes,2001,6.1,25854,"['action', 'crime', 'drama', 'thriller']"
3,m3,2001: a space odyssey,1968,8.4,163227,"['adventure', 'mystery', 'sci-fi']"
4,m4,48 hrs.,1982,6.9,22289,"['action', 'comedy', 'crime', 'drama', 'thrill..."


In [57]:
mtm_df.to_csv(mtm_csv, sep=',')
print("Saved to csv..")

Saved to csv..


In [58]:
# Movie Characters Metadata.txt
mcm = 'cornell movie-dialogs corpus/movie_characters_metadata.txt'
# CSV for transforming the data
mcm_csv = 'mcm_csv.csv'

In [59]:
# Add header rows
with open(mcm_csv, 'wt') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames = ["Character_Id", "Character_Name", "Movie_Number", 
                                                   "Movie_Title", "Gender", "Position_in_Credits"])
    writer.writeheader()

In [60]:
# Open the second .txt file and write it onto the next dataframe
with open(mcm, 'r') as mcmeta:
    for lines in mcmeta:
        text = str(lines)
        line_list = text.split('+++$+++')

        with open(mcm_csv, 'a') as write_csv:
            writer = csv.writer(write_csv)
            writer.writerow([line_list[0].strip(), line_list[1].strip(), line_list[2].strip(),
                            line_list[3].strip(), line_list[4].strip(), line_list[5].strip()])

In [61]:
mcm_df = pd.read_csv(mcm_csv)
print(mcm_df.shape)
mcm_df.head(5)

(9035, 6)


,Character_Id,Character_Name,Movie_Number,Movie_Title,Gender,Position_in_Credits
0,u0,BIANCA,m0,10 things i hate about you,f,4
1,u1,BRUCE,m0,10 things i hate about you,?,?
2,u2,CAMERON,m0,10 things i hate about you,m,3
3,u3,CHASTITY,m0,10 things i hate about you,?,?
4,u4,JOEY,m0,10 things i hate about you,m,6


In [62]:
mcm_df.to_csv(mcm_csv, sep=',')
print("Saved to csv..")

Saved to csv..


In [63]:
# Movie Characters Metadata.txt
ml = 'cornell movie-dialogs corpus/movie_lines.txt'
# CSV for transforming the data
ml_csv = 'ml_csv.csv'

In [64]:
# Add header rows
with open(ml_csv, 'wt') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames = ["Line_Id", "Character_Id", "Movie_Number", 
                                                   "Character_Name", "Dialogue"])
    writer.writeheader()

In [65]:
# Open the second .txt file and write it onto the next dataframe
with open(ml, 'r') as mlines:
    for lines in mlines:
        text = str(lines)
        line_list = text.split('+++$+++')

        with open(ml_csv, 'a') as write_csv:
            writer = csv.writer(write_csv)
            writer.writerow([line_list[0].strip(), line_list[1].strip(), line_list[2].strip(),
                            line_list[3].strip(), line_list[4].strip()])

In [66]:
ml_df = pd.read_csv('ml_csv.csv')
print(ml_df.shape)
ml_df.head(5)

(304713, 5)


,Line_Id,Character_Id,Movie_Number,Character_Name,Dialogue
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.


In [51]:
ml_df.to_csv(ml_csv, sep = ',')
print('Saved to csv...')

Saved to csv...
